In [1]:
import pandas as pd
import sqlite3

In [2]:
# Load CSV data into DataFrames
transactions_df = pd.read_csv('data/transaction_tab.csv')
accounts_df = pd.read_csv('data/account_tab.csv')

# Display the first few rows of each DataFrame
display(transactions_df.head())
display(accounts_df.head())

,account_id,transaction_id,transaction_date,transaction_type,amount
0,520,27,2019-04-01,transfer,20000
1,482,28,2019-04-01,card,75000
2,241,30,2019-04-02,transfer,15000
3,297,31,2019-04-02,card,10000
4,145,32,2019-04-03,transfer,10000


,account_id,country_id,created_at
0,1,198,2019-02-22 16:31:20
1,2,101,2019-02-22 16:46:41
2,3,198,2019-02-22 18:09:36
3,4,198,2019-02-22 22:17:04
4,5,198,2019-02-22 22:37:56


In [3]:
# Connect to SQLite database (or create it)
db_path = 'data/transactions.db'
conn = sqlite3.connect(db_path)

# Write the data into SQLite database
transactions_df.to_sql('transactions', conn, if_exists='replace', index=False)
accounts_df.to_sql('accounts', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [4]:
# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# SQL query for adoption
adoption_query = """
WITH first_transactions AS (
  SELECT 
    account_id,
    strftime('%Y-%m', transaction_date) AS transaction_month,
    transaction_type,
    ROW_NUMBER() OVER (
      PARTITION BY account_id, transaction_type
      ORDER BY transaction_date
    ) AS rn
  FROM transactions
)
SELECT 
  transaction_month,
  transaction_type,
  COUNT(DISTINCT account_id) AS new_adopters
FROM first_transactions
WHERE rn = 1
GROUP BY transaction_month, transaction_type
ORDER BY transaction_month, transaction_type;
"""

# Execute the query
cursor.execute(adoption_query)

# Fetch and display the results
adoption_results = cursor.fetchall()
adoption_df = pd.DataFrame(
    adoption_results, columns=["month", "transaction_type", "adoption_count"]
)
display(adoption_df)

# Close the connection
conn.close()

,month,transaction_type,adoption_count
0,2019-04,card,19
1,2019-04,transfer,23
2,2019-05,card,36
3,2019-05,transfer,33
4,2019-06,card,49
5,2019-06,transfer,52


In [5]:
# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# SQL query for usage
usage_query = """
WITH RECURSIVE date_series AS (
    SELECT DATE(MIN(transaction_date)) AS date
    FROM transactions
    UNION ALL
    SELECT DATE(date, '+1 day')
    FROM date_series
    WHERE DATE(date, '+1 day') <= (
            SELECT DATE(MAX(transaction_date))
            FROM transactions
        )
),
active_accounts AS (
    SELECT ds.date AS report_date,
        t.account_id
    FROM date_series ds
        JOIN transactions t ON DATE(t.transaction_date) BETWEEN DATE(ds.date, '-44 days') AND ds.date
    WHERE t.transaction_type = 'transfer'
    GROUP BY ds.date,
        t.account_id
    HAVING COUNT(DISTINCT t.transaction_id) >= 2
)
SELECT strftime('%Y-%m', report_date) AS month,
    COUNT(DISTINCT account_id) AS active_account_count
FROM active_accounts
GROUP BY strftime('%Y-%m', report_date)
ORDER BY month;
"""

# Execute the query
cursor.execute(usage_query)

# Fetch and display the results
usage_results = cursor.fetchall()
usage_df = pd.DataFrame(usage_results, columns=['month', 'active_account_count'])
display(usage_df)

# Close the connection
conn.close()

,month,active_account_count
0,2019-04,1
1,2019-05,6
2,2019-06,11
